In [4]:

import pandas as pd
import glob
import os

# Path to your folder
path = "/Users/rem76/PycharmProjects/TLOmodel/resources/climate_change_impacts/"

# Find all relevant CSVs
files = glob.glob(os.path.join(path, "ResourceFile_Precipitation_Disruptions_*.csv"))

all_values = []

for f in files:
    df = pd.read_csv(f)
    if "mean_all_service" in df.columns:
        all_values.extend(df["mean_all_service"].dropna().tolist()) 

all_values_series = pd.Series(all_values)

# Compute summary across all files
summary_all_files = {
    "min": all_values_series.min(),
    "max": all_values_series.max(),
    "mean": all_values_series.mean()
}

print(summary_all_files)


{'min': 0.0, 'max': 1.5805192824391343, 'mean': 0.011370392341963076}


In [2]:
files

[]